<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT%2BResNeXt%2BImage_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.7
!pip install torch transformers torchvision

In [ ]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
'''
IMPORTING NECESSARY MODULES
'''

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# sys.path.append('./Trainers/')
# sys.path.append('./Dataloaders/')
# sys.path.append('./utils/')
# sys.path.append('./architectures/')

sys.path.append('/content/gdrive/MyDrive/hm_project_pe/data/Trainers/')
sys.path.append('/content/gdrive/MyDrive/hm_project_pe/data/Dataloaders/')
sys.path.append('/content/gdrive/MyDrive/hm_project_pe/data/utils/')
sys.path.append('/content/gdrive/MyDrive/hm_project_pe/data/architectures/')
sys.path.append('/content/gdrive/MyDrive/hm_project_pe/data/Test/')

from dataloader import mydataset_captioning, mytestdataset
from Load_model import load
from plot_curves import plot_loss, plot_acc
from trainer_Resnet_BERT_Captioning import train, test_classify

'''
For ResNeXt
'''
from resnet_models import ResNet,Bottleneck, resnext101_32x8d

'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

**Device**

In [ ]:
# gpu_ids = [7,6]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


**Dataloading Scheme**

In [ ]:
trainlist = '/content/gdrive/MyDrive/hm_project_pe/data/train.jsonl'
validlist = '/content/gdrive/MyDrive/hm_project_pe/data/dev.jsonl'
img_dir = '/content/gdrive/MyDrive/hm_project_pe/data'

In [ ]:
'''
Train Dataloader
''' 
train_dataset = mydataset_captioning(annotations_file=trainlist, img_dir=img_dir, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 32, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset_captioning(annotations_file=validlist, img_dir=img_dir, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

**Model Definition**

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
'''
Model1 ResNeXt101_32x8d
'''
# Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)
Image_model = resnext101_32x8d()

Image_model.fc = nn.Sequential(
    nn.Linear(Image_model.fc.in_features, 2)
    )

Image_model = nn.DataParallel(Image_model,device_ids = [0]).to(device)


'''
Load saved model from checkpoint
'''
model1_name = 'ResneXt101_32x8d'
model1_path = './saved_model_checkpoints/'+model1_name

checkpoint1 = torch.load(model1_path)
Image_model.load_state_dict(checkpoint1['model_state_dict'])
Image_model.to(device)

In [ ]:
'''
Model 2 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model = nn.DataParallel(Text_model,device_ids=[0]).to(device)


'''
Load saved model from checkpoint
'''
model2_name = 'BERT_basic'
model2_path = './saved_model_checkpoints/'+model2_name

checkpoint2 = torch.load(model2_path)
Text_model.load_state_dict(checkpoint2['model_state_dict'])

Text_model.to(device)

In [ ]:
'''
Fusion 
Image Features, Text Features and Captions generated by our Captioning model
'''
class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.concat = nn.Linear(in_features=768+2048+768, out_features= 512)
        
        self.bn = nn.BatchNorm1d(512)
        self.bn1 = nn.BatchNorm1d(768)
        self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(768)

        
        self.dropout = nn.Dropout(drop_prob)
        
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features, image_features, caption_features):

        text_features = self.bn1(text_features)
        image_features = self.bn2(image_features)
        caption_features = self.bn3(caption_features)

        fused_input =  torch.cat((text_features, image_features, caption_features), dim=1)
        
        x = self.concat(fused_input)
        x = F.relu(self.bn(x))        
        
        x = F.relu(self.classify(x)) 

        return x


In [ ]:
Fusion_model = FusionNet(num_classes = 2 , drop_prob = 0.1)
Fusion_model = nn.DataParallel(Fusion_model, device_ids=[0]).to(device)

**Hyperparameters**

In [ ]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)
# optimizer = AdamW(Fusion_model.parameters(), lr = 2e-3, eps = 1e-8)


'''
Number of training epochs.
'''
num_Epochs = 10


# '''
# OneCycleLR
# '''
# max_lr = 0.05
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, last_epoch=-1)


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.001)

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

In [ ]:
model_name = 'ImageCaptioning'
model_path = '.saved_model_checkpoints/'+model_name

In [ ]:
writer = SummaryWriter(model_name)

train(Image_model, Text_model, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

In [ ]:
'''
Load saved model from checkpoint
'''
Fusion_model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(model_path, Fusion_model, optimizer, lr_scheduler)


**Evaluate**

In [ ]:
test_classify(Fusion_model, validation_dataloader, criterion, device)

**Predict on Test and generate output.csv**

**Test Dataloader**

In [ ]:
testlist = 'path of test set'

test_dataset = mytestdataset(testlist, name='test')          
test_dataloader = data.DataLoader(test_dataset, shuffle= False, batch_size = 32, num_workers=8,pin_memory=True)

In [ ]:
from predict import predict
predict(image_model, text_model, fusion_model, test_dataloader, device)